<table align="left">
  <tr>
    <td><img src="fleet.png" alt="fleet of icecream trucks" width="120"/></td>
    <td align="left"><h1>Lesson 4: Monitoring and Evaluating your Agent</h1></td>
  </tr>
</table>


<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> file:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

## Setup Tracing

In [1]:
PROJECT_NAME = "Customer-Success"

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
from phoenix.otel import register
from openinference.instrumentation.smolagents import SmolagentsInstrumentor

tracer_provider = register(
    project_name=PROJECT_NAME,
    #endpoint= get_phoenix_endpoint() + "v1/traces"
    endpoint = os.getenv('DLAI_LOCAL_URL').format(port='6006') + "v1/traces"
)
SmolagentsInstrumentor().instrument(tracer_provider=tracer_provider)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: Customer-Success
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://s172-29-8-74p6006.lab-aws-production.deeplearning.ai/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [3]:
from dotenv import load_dotenv, find_dotenv
load_dotenv() # load variables from local .env file

from huggingface_hub import login

login(os.getenv('HF_API_KEY'))

In [4]:
from smolagents import HfApiModel

model=HfApiModel("Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

model([{"role": "user", "content": "Hello!"}])

ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='Hello! How can I assist you today?', tool_calls=[], raw=ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop', index=0, message=ChatCompletionOutputMessage(role='assistant', content='Hello! How can I assist you today?', tool_call_id=None, tool_calls=[]), logprobs=None, seed=16112240548779254000)], created=1745936843, id='ns3Th4B-4yUbBN-937f7ad7a8f4c994', model='Qwen/Qwen2.5-Coder-32B-Instruct', system_fingerprint=None, usage=ChatCompletionOutputUsage(completion_tokens=10, prompt_tokens=31, total_tokens=41), object='chat.completion', prompt=[]))

In [5]:
# This is where you can access the display:
print(os.environ.get('DLAI_LOCAL_URL').format(port='6006'))

https://s172-29-8-74p6006.lab-aws-production.deeplearning.ai/


## Trace an agent run

In [6]:
from smolagents import HfApiModel, CodeAgent

agent = CodeAgent(model=model, tools=[])

>Note, the following line will sometimes get a timeout on the interface to the tracing package due to the networked interface. If this happens, try it again.


In [7]:
agent.run("What is the 100th Fibonacci number?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the 100th Fibonacci number?                                                                             │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  def fibonacci(n):                                                                                                
      a, b = 0, 1                                                                                                  
      for _ in range(n):                                                                                           
          a, b = b, a + b                                                                                          
      return a                                                                                                     
                                                                                                                   
  fib_100 = fibonacci(100)                                                                                         
  final_answer(fib_100)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 354224848179261915075

[Step 1: Duration 0.12 seconds| Input tokens: 2,020 | Output tokens: 130]

354224848179261915075

In [8]:
# This is where you can access the display:
print(os.environ.get('DLAI_LOCAL_URL').format(port='6006'))

https://s172-29-8-74p6006.lab-aws-production.deeplearning.ai/


## Setup ice cream production system

In [9]:
from smolagents import tool
from typing import Dict

menu_prices = {"crepe nutella": 1.50, "vanilla ice cream": 2, "maple pancake": 1.}

ORDER_BOOK = {}

@tool
def place_order(quantities: Dict[str, int], session_id: int) -> None:
    """Places a pre-order of snacks.

    Args:
        quantities: a dictionary with names as keys and quantities as values
        session_id: the id for the client session
    """
    global ORDER_BOOK
    assert isinstance(quantities, dict), "Incorrect type for the input dictionary!"
    assert [key in menu_prices for key in quantities.keys()], f"All food names should be within {menu_prices.keys()}"
    ORDER_BOOK[session_id] = quantities

@tool
def get_prices(quantities: Dict[str, int]) -> str:
    """Gets price for certain quantities of ice cream.

    Args:
        quantities: a dictionary with names as keys and quantities as values
    """
    assert isinstance(quantities, dict), "Incorrect type for the input dictionary!"
    assert [key in menu_prices for key in quantities.keys()], f"All food names should be within {menu_prices.keys()}"
    total_price = sum([menu_prices[key] * value for key, value in quantities.items()])
    return (
        f"Given the current menu prices:\n{menu_prices}\nThe total price for your order would be: ${total_price}"
    )

In [10]:
order_agent = CodeAgent(
    tools=[place_order, get_prices],
    model=HfApiModel("Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")
)

In [11]:
order_agent.run(
    "Could I come and collect one crepe nutella?",
    additional_args={"session_id": 192}
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could I come and collect one crepe nutella?                                                                     │
│ You have been provided with these additional arguments, that you can access using the keys as variables in your │
│ python code:                                                                                                    │
│ {'session_id': 192}.                                                                                            │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  place_order(quantities={'crepe nutella': 1}, session_id=session_id)                                              
  final_answer("Order placed for one crepe nutella.")                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Order placed for one crepe nutella.

[Step 1: Duration 0.11 seconds| Input tokens: 2,193 | Output tokens: 68]

'Order placed for one crepe nutella.'

### Try multiple orders

In [12]:
client_requests = [
    ("Could I come and collect one crepe nutella?", "place_order"),
    ("What would be the price for 1 crêpe nutella + 2 pancakes?", "get_prices"),
    ("How did you start your ice-cream business?", None),
    ("What's the weather at the Louvre right now?", None),
    ("I'm not sure if I should order. I want a vanilla ice cream. but if it's more expensive than $1, I don't want it. If it's below, I'll order it, please.", "place_order")
]

In [13]:
for request in client_requests:
    order_agent.run(
        request[0],
        additional_args={"session_id": 0, "menu_prices": menu_prices}
    )

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could I come and collect one crepe nutella?                                                                     │
│ You have been provided with these additional arguments, that you can access using the keys as variables in your │
│ python code:                                                                                                    │
│ {'session_id': 0, 'menu_prices': {'crepe nutella': 1.5, 'vanilla ice cream': 2, 'maple pancake': 1.0}}.         │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  quantities = {'crepe nutella': 1}                                                                                
  place_order(quantities=quantities, session_id=session_id)                                                        
  final_answer("Order placed for one crepe nutella.")                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Order placed for one crepe nutella.

[Step 1: Duration 0.12 seconds| Input tokens: 2,227 | Output tokens: 93]

...

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ I'm not sure if I should order. I want a vanilla ice cream. but if it's more expensive than $1, I don't want    │
│ it. If it's below, I'll order it, please.                                                                       │
│ You have been provided with these additional arguments, that you can access using the keys as variables in your │
│ python code:                                                                                                    │
│ {'session_id': 0, 'menu_prices': {'crepe nutella': 1.5, 'vanilla ice cream': 2, 'maple pancake': 1.0}}.         │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  vanilla_ice_cream_price = menu_prices['vanilla ice cream']                                                       
  print(f"The price of vanilla ice cream is ${vanilla_ice_cream_price}.")                                          
                                                                                                                   
  final_answer("I will not order the vanilla ice cream as it is more than $1.")                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
The price of vanilla ice cream is $2.

Out - Final answer: I will not order the vanilla ice cream as it is more than $1.

[Step 1: Duration 0.09 seconds| Input tokens: 2,260 | Output tokens: 162]

In [14]:
import phoenix as px

spans = px.Client().get_spans_dataframe(project_name=PROJECT_NAME)
spans.head(20)

/usr/local/lib/python3.11/site-packages/phoenix/utilities/client.py:60: UserWarning: The Phoenix server (8.31.0) and client (8.19.1) versions are mismatched and may have compatibility issues.
  warnings.warn(


,name,span_kind,parent_id,start_time,end_time,status_code,status_message,events,context.span_id,context.trace_id,...,attributes.llm.invocation_parameters,attributes.llm.output_messages,attributes.output.value,attributes.input.value,attributes.llm.token_count.prompt,attributes.llm.input_messages,attributes.tool.parameters,attributes.tool.description,attributes.tool.name,attributes.smolagents
context.span_id,,,,,,,,,,,,,,,,,,,,,
e42ce435bbf6fe35,HfApiModel.__call__,LLM,None,2025-05-02 18:24:04.179909+00:00,2025-05-02 18:24:04.337494+00:00,OK,,[],e42ce435bbf6fe35,44ad68d042cbe745783f75a5eb673753,...,{},"[{'message.role': 'assistant', 'message.conten...","{""role"": ""assistant"", ""content"": ""Hello! How c...","{""messages"": [{""role"": ""user"", ""content"": ""Hel...",31.0,None,None,None,None,None
b097cdc24699a38a,HfApiModel.__call__,LLM,9980addc1c8b681b,2025-05-02 18:24:04.449087+00:00,2025-05-02 18:24:04.467324+00:00,OK,,[],b097cdc24699a38a,6725f250ce2fcd0b88c5453615364fd2,...,{},"[{'message.role': 'assistant', 'message.conten...","{""role"": ""assistant"", ""content"": ""Thought: To ...","{""messages"": [{""role"": ""system"", ""content"": [{...",2020.0,"[{'message.role': 'system', 'message.content':...",None,None,None,None
747b923a9b7b99f6,FinalAnswerTool,TOOL,9980addc1c8b681b,2025-05-02 18:24:04.518317+00:00,2025-05-02 18:24:04.518384+00:00,OK,,[],747b923a9b7b99f6,6725f250ce2fcd0b88c5453615364fd2,...,None,None,None,"{""args"": [354224848179261915075], ""sanitize_in...",NaN,None,"{'answer': {'type': 'any', 'description': 'The...",Provides a final answer to the given problem.,final_answer,None
9980addc1c8b681b,Step 1,CHAIN,2019bfb4bf154f3f,2025-05-02 18:24:04.448857+00:00,2025-05-02 18:24:04.543453+00:00,OK,,[],9980addc1c8b681b,6725f250ce2fcd0b88c5453615364fd2,...,None,None,Execution logs:\nLast output from code snippet...,"{""memory_step"": ""ActionStep(model_input_messag...",NaN,None,None,None,None,None
2019bfb4bf154f3f,CodeAgent.run,AGENT,None,2025-05-02 18:24:04.439826+00:00,2025-05-02 18:24:04.567290+00:00,OK,,[],2019bfb4bf154f3f,6725f250ce2fcd0b88c5453615364fd2,...,None,None,354224848179261915075,"{""task"": ""What is the 100th Fibonacci number?""...",2020.0,None,None,None,None,"{'tools_names': ['final_answer'], 'max_steps':..."
86e2ad2398b88552,HfApiModel.__call__,LLM,dc804ca4bcd35b26,2025-05-02 18:24:04.622366+00:00,2025-05-02 18:24:04.639981+00:00,OK,,[],86e2ad2398b88552,37565150494cea5f90ea86918bf88d93,...,{},"[{'message.role': 'assistant', 'message.conten...","{""role"": ""assistant"", ""content"": ""Thought: I n...","{""messages"": [{""role"": ""system"", ""content"": [{...",2193.0,"[{'message.role': 'system', 'message.content':...",None,None,None,None
0dc828ce0004c1e0,SimpleTool,TOOL,dc804ca4bcd35b26,2025-05-02 18:24:04.664048+00:00,2025-05-02 18:24:04.664103+00:00,OK,,[],0dc828ce0004c1e0,37565150494cea5f90ea86918bf88d93,...,None,None,None,"{""args"": [], ""sanitize_inputs_outputs"": false,...",NaN,None,"{'quantities': {'type': 'object', 'additionalP...",Places a pre-order of snacks.,place_order,None
6fd369fccd2f55ac,FinalAnswerTool,TOOL,dc804ca4bcd35b26,2025-05-02 18:24:04.687383+00:00,2025-05-02 18:24:04.687439+00:00,OK,,[],6fd369fccd2f55ac,37565150494cea5f90ea86918bf88d93,...,None,None,None,"{""args"": [""Order placed for one crepe nutella....",NaN,None,"{'answer': {'type': 'any', 'description': 'The...",Provides a final answer to the given problem.,final_answer,None
dc804ca4bcd35b26,Step 1,CHAIN,dfb259d11a079c04,2025-05-02 18:24:04.622141+00:00,2025-05-02 18:24:04.708569+00:00,OK,,[],dc804ca4bcd35b26,37565150494cea5f90ea86918bf88d93,...,None,None,Execution logs:\nLast output from code snippet...,"{""memory_step"": ""ActionStep(model_input_messag...",NaN,None,None,None,None,None


### Add processing to extract desired information

In [15]:
import pandas as pd
import json

agents = spans[spans['span_kind'] == 'AGENT'].copy()
agents['task'] = agents['attributes.input.value'].apply(
    lambda x: json.loads(x).get('task') if isinstance(x, str) else None
)

tools = spans.loc[
    spans['span_kind'] == 'TOOL',
    ["attributes.tool.name", "attributes.input.value", "context.trace_id"]
].copy()

tools_per_task = agents[
    ["name", "start_time", "task", "context.trace_id"]
].merge(
    tools,
    on="context.trace_id",
    how="left",
)
tools_per_task.head()

,name,start_time,task,context.trace_id,attributes.tool.name,attributes.input.value
0,CodeAgent.run,2025-05-02 18:24:04.439826+00:00,What is the 100th Fibonacci number?,6725f250ce2fcd0b88c5453615364fd2,final_answer,"{""args"": [354224848179261915075], ""sanitize_in..."
1,CodeAgent.run,2025-05-02 18:24:04.616302+00:00,Could I come and collect one crepe nutella?,37565150494cea5f90ea86918bf88d93,place_order,"{""args"": [], ""sanitize_inputs_outputs"": false,..."
2,CodeAgent.run,2025-05-02 18:24:04.616302+00:00,Could I come and collect one crepe nutella?,37565150494cea5f90ea86918bf88d93,final_answer,"{""args"": [""Order placed for one crepe nutella...."
3,CodeAgent.run,2025-05-02 18:24:04.766269+00:00,Could I come and collect one crepe nutella?,29de4090aeba0c956d1c489e4f329ca4,place_order,"{""args"": [], ""sanitize_inputs_outputs"": false,..."
4,CodeAgent.run,2025-05-02 18:24:04.766269+00:00,Could I come and collect one crepe nutella?,29de4090aeba0c956d1c489e4f329ca4,final_answer,"{""args"": [""Order placed for one crepe nutella...."


### Now, compare tool calls with exected tool calls

In [16]:
def score_request(expected_tool: str, tool_calls: list):
    if expected_tool is None:
        return tool_calls == set(["final_answer"])
    else:
        return expected_tool in tool_calls

results = []
for request, expected_tool in client_requests:
    tool_calls = set(tools_per_task.loc[tools_per_task["task"] == request, "attributes.tool.name"].tolist())
    results.append(
        {
            "request": request,
            "tool_calls_performed": tool_calls,
            "is_correct": score_request(expected_tool, tool_calls)
        }
    )
pd.DataFrame(results)

,request,tool_calls_performed,is_correct
0,Could I come and collect one crepe nutella?,"{final_answer, place_order}",True
1,What would be the price for 1 crêpe nutella + ...,"{final_answer, get_prices}",True
2,How did you start your ice-cream business?,"{final_answer, get_prices}",False
3,What's the weather at the Louvre right now?,{final_answer},True
4,I'm not sure if I should order. I want a vanil...,{},False
